In [1]:
import pandas as pd
import numpy as np
from datetime import date
import datetime as DT
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import pickle
import re

In [2]:
!pip install pyenchant

In [3]:
df = pd.read_csv('Input.csv')

In [4]:
df=df.rename(columns={'Mobile':'phone_numbers'})
df=df.rename(columns={'Reg Client IP':'ip'})

In [5]:
pd.options.display.max_columns=100
df.head()

,FirstName,MiddleName,LastName,Dob,Gender,UserId,Status,Email,phone_numbers,ProgramId,ProgramName,PostalCode,BadTryCount,LastBadLoginTryTime,LastLoginTime,LastPinChangeTime,PresentLoginTime,RegisteredOn,OS Type,Reg Referral Code,Reg Referral Prefix,Ref PC AC Number,ip,Device Id,Sim Id,Meta Data
0,Krishna,Nagnathrao,Patil,xxxx0908,U,255xx240,1,NaN,9707622140,6019,PayZapp,431717.0,0,NaN,14:44.0,NaN,35:21.0,01-11-2020 00:00,Android,NaN,NaN,NaN,49.35.221.2,anid:INE9EIU2dobAh3xITmtgvkw7mck8OVKbztVHLRW2R...,anid:INE9EIU2dobAh3xITmtgvkw7mck8OVKbztVHLRW2R...,7eba668f-d06e-492c-b008-f4b169492b1c
1,Jayasri,NaN,Kota,xxxx0913,U,255xx148,1,NaN,9464191867,6019,PayZapp,521001.0,0,41:12.0,42:07.0,41:56.0,59:06.0,01-11-2020 00:01,Android,NaN,NaN,NaN,117.197.223.15,anid:ZipdGGO206z2mOdhJ/PkUe6Bzp4fmjZa28MZZA3Rg...,anid:ZipdGGO206z2mOdhJ/PkUe6Bzp4fmjZa28MZZA3Rg...,d1499c8d-cbe9-48ae-b40e-963ca0898b60
2,Saikat,NaN,Saha,xxxx0516,M,255xx242,1,ssxxxikxxxtsxxxhxxx1996@gmail.com,9432504977,6019,PayZapp,700027.0,0,NaN,12:09.0,NaN,02:49.0,01-11-2020 00:02,Android,NaN,NaN,NaN,182.66.181.8,anid:/7NwY1Rxm5AW3kdfGXYZkfUSLYedudx0w1uynHVk9...,anid:/7NwY1Rxm5AW3kdfGXYZkfUSLYedudx0w1uynHVk9...,19a86cdf-aeaf-4030-9abc-5fff6346480c
3,rahul,NaN,Verma,xxxx0225,U,255xx696,1,NaN,7074212592,6019,PayZapp,184205.0,0,NaN,59:05.0,NaN,48:31.0,01-11-2020 00:03,Android,NaN,NaN,NaN,205.253.177.88,anid:cm0e88QJqfwMxBC7KOKxKdOkT3GirG8maWuBPBFol...,anid:cm0e88QJqfwMxBC7KOKxKdOkT3GirG8maWuBPBFol...,6baa013f-7566-4f8f-9c7c-89d9c5117f84
4,Sujit,NaN,Jha,xxxx0203,U,255xx974,1,NaN,9413346337,6019,PayZapp,NaN,0,NaN,09:27.0,NaN,58:20.0,01-11-2020 00:04,NaN,NaN,NaN,NaN,103.225.177.108,NaN,NaN,NaN


In [6]:
df.shape

(1485, 26)

### Check the nulls

In [7]:
df.isnull().sum()

FirstName                 0
MiddleName             1238
LastName                  7
Dob                       0
Gender                    0
UserId                    0
Status                    0
Email                  1209
phone_numbers             0
ProgramId                 0
ProgramName               0
PostalCode               24
BadTryCount               0
LastBadLoginTryTime    1367
LastLoginTime           415
LastPinChangeTime      1409
PresentLoginTime         18
RegisteredOn              0
OS Type                  24
Reg Referral Code      1307
Reg Referral Prefix    1481
Ref PC AC Number       1312
ip                        0
Device Id                24
Sim Id                   24
 Meta Data              139
dtype: int64

In [8]:
df['phone_numbers']= df['phone_numbers'].astype(str)

In [9]:
df.shape

(1485, 26)

### Gibberish names in the dataset 

In [10]:
with open('unigrams_model.pickle', 'rb') as f:
    unigrams = pickle.load(f)

In [11]:
with open('bigrams_model.pickle', 'rb') as f:
    bigrams = pickle.load(f)

In [12]:
with open('trigrams_model.pickle', 'rb') as f:
    trigrams = pickle.load(f)

In [13]:
weights = [0.001, 0.01, 0.989]

def strangeness(text): 
    r = 0   
    text = '  ' + text + '\n'  
    for i in range(2, len(text)):
        char = text[i]
        context1 = text[(i-1):i]
        context2 = text[(i-2):i]
        num = unigrams[char] * weights[0] + bigrams[context1+char] * weights[1] + trigrams[context2+char] * weights[2] 
        den = sum(unigrams.values()) * weights[0] + unigrams[char] + weights[1] + bigrams[context1] * weights[2]
        r = r- np.log(num / den)
    return r / (len(text) - 2)

In [14]:
df['FirstNameLower'] = df['FirstName'].str.lower()
df['MiddleNameLower'] = df['MiddleName'].str.lower()
df['LastNameLower'] = df['LastName'].str.lower()

In [15]:
df['FullName'] = df['FirstNameLower'].fillna('') + ' ' +  df['MiddleNameLower'].fillna('') + ' ' + df['LastNameLower'].fillna('')

In [16]:
df['FullName'] = df['FullName'].map(lambda x: re.sub(r'[^a-z ]', '', x))

In [17]:
df['Score'] = df['FullName'].apply(lambda x:strangeness(x))

In [18]:
#df = df.sort_values(by = 'Score')

In [19]:
df['IsGibberish'] = np.where(df['Score'] >= 6, 1, 0)

In [20]:
df['FirstNameLen'] = df['FirstNameLower'].astype(str).map(len)
df['MiddleNameLen'] = df['MiddleNameLower'].astype(str).map(len)
df['LastNameLen'] = df['LastNameLower'].astype(str).map(len)

In [21]:
df['FirstNameLen'] = np.where(df['FirstNameLower'].isna(), 0, df['FirstNameLen'])
df['MiddleNameLen'] = np.where(df['MiddleNameLower'].isna(), 0, df['MiddleNameLen'])
df['LastNameLen'] = np.where(df['LastNameLower'].isna(), 0, df['LastNameLen'])
df['TotalNameLength'] = df['FirstNameLen'] + df['MiddleNameLen'] + df['LastNameLen']

In [22]:
#df['IsGibberish'] = np.where(df['TotalNameLength'] < 5, 1, df['IsGibberish'])

In [23]:
import enchant
d = enchant.Dict("en_US")

In [24]:
letter_list = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i','j', 'k', 'l', 'm', 'n',
              'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [25]:
common_indian_last_names = ['kumar', 'singh', 'khan', 'raj', 'pal', 'shah', 'sen', 'ram','gill','goel']

In [26]:
def get_english_words_count(full_name):
    english_word_counter = 0
    for name in full_name:
        is_english_word = d.check(name)
        if is_english_word is True and name not in letter_list and name not in common_indian_last_names:
            english_word_counter = english_word_counter + 1
    return english_word_counter

In [27]:
def get_english_words(full_name):
    english_words_found = []
    for name in full_name:
        is_english_word = d.check(name)
        if is_english_word is True and name not in letter_list and name not in common_indian_last_names:
            english_words_found.append(name)
    return english_words_found

In [28]:
df['EnglishWordsCount'] = df['FullName'].apply(lambda x: get_english_words_count(x.split()))

In [29]:
df['EnglishWords'] = df['FullName'].apply(lambda x: get_english_words(x.split()))

In [30]:
df[df['EnglishWordsCount'].isin([1,2,3,4])].to_csv('names_with_english_words.csv', index = False)

In [31]:
df['TotalWordsCount'] = df['FullName'].apply(lambda x: len(x.split()))

In [32]:
df['PctEnglishWordsCount'] = df['EnglishWordsCount']/df['TotalWordsCount']

In [33]:
df['Vowels'] = df.FullName.str.lower().str.count(r'[aeiou]')

In [34]:
df['IsGibberish'] = np.where((df['Score'] > 5) & (df['EnglishWordsCount'] > 1) & (df['PctEnglishWordsCount'] >= 0.5), 1, df['IsGibberish'])
df['IsGibberish'] = np.where((df['Score'] < 5.9) & (df['TotalWordsCount'] <= 2) & (df['Vowels'] > 0), 0, df['IsGibberish'])
df['IsGibberish'] = np.where((df['Vowels'] == 0), 1, df['IsGibberish'])
df['IsGibberish'] = np.where((df['EnglishWordsCount'] > 1), 1, df['IsGibberish'])

In [35]:
df.head()

,FirstName,MiddleName,LastName,Dob,Gender,UserId,Status,Email,phone_numbers,ProgramId,ProgramName,PostalCode,BadTryCount,LastBadLoginTryTime,LastLoginTime,LastPinChangeTime,PresentLoginTime,RegisteredOn,OS Type,Reg Referral Code,Reg Referral Prefix,Ref PC AC Number,ip,Device Id,Sim Id,Meta Data,FirstNameLower,MiddleNameLower,LastNameLower,FullName,Score,IsGibberish,FirstNameLen,MiddleNameLen,LastNameLen,TotalNameLength,EnglishWordsCount,EnglishWords,TotalWordsCount,PctEnglishWordsCount,Vowels
0,Krishna,Nagnathrao,Patil,xxxx0908,U,255xx240,1,NaN,9707622140,6019,PayZapp,431717.0,0,NaN,14:44.0,NaN,35:21.0,01-11-2020 00:00,Android,NaN,NaN,NaN,49.35.221.2,anid:INE9EIU2dobAh3xITmtgvkw7mck8OVKbztVHLRW2R...,anid:INE9EIU2dobAh3xITmtgvkw7mck8OVKbztVHLRW2R...,7eba668f-d06e-492c-b008-f4b169492b1c,krishna,nagnathrao,patil,krishna nagnathrao patil,4.857123,0,7,10,5,22,0,[],3,0.0,8
1,Jayasri,NaN,Kota,xxxx0913,U,255xx148,1,NaN,9464191867,6019,PayZapp,521001.0,0,41:12.0,42:07.0,41:56.0,59:06.0,01-11-2020 00:01,Android,NaN,NaN,NaN,117.197.223.15,anid:ZipdGGO206z2mOdhJ/PkUe6Bzp4fmjZa28MZZA3Rg...,anid:ZipdGGO206z2mOdhJ/PkUe6Bzp4fmjZa28MZZA3Rg...,d1499c8d-cbe9-48ae-b40e-963ca0898b60,jayasri,NaN,kota,jayasri kota,5.010674,0,7,0,4,11,0,[],2,0.0,5
2,Saikat,NaN,Saha,xxxx0516,M,255xx242,1,ssxxxikxxxtsxxxhxxx1996@gmail.com,9432504977,6019,PayZapp,700027.0,0,NaN,12:09.0,NaN,02:49.0,01-11-2020 00:02,Android,NaN,NaN,NaN,182.66.181.8,anid:/7NwY1Rxm5AW3kdfGXYZkfUSLYedudx0w1uynHVk9...,anid:/7NwY1Rxm5AW3kdfGXYZkfUSLYedudx0w1uynHVk9...,19a86cdf-aeaf-4030-9abc-5fff6346480c,saikat,NaN,saha,saikat saha,4.904884,0,6,0,4,10,0,[],2,0.0,5
3,rahul,NaN,Verma,xxxx0225,U,255xx696,1,NaN,7074212592,6019,PayZapp,184205.0,0,NaN,59:05.0,NaN,48:31.0,01-11-2020 00:03,Android,NaN,NaN,NaN,205.253.177.88,anid:cm0e88QJqfwMxBC7KOKxKdOkT3GirG8maWuBPBFol...,anid:cm0e88QJqfwMxBC7KOKxKdOkT3GirG8maWuBPBFol...,6baa013f-7566-4f8f-9c7c-89d9c5117f84,rahul,NaN,verma,rahul verma,4.695897,0,5,0,5,10,0,[],2,0.0,4
4,Sujit,NaN,Jha,xxxx0203,U,255xx974,1,NaN,9413346337,6019,PayZapp,NaN,0,NaN,09:27.0,NaN,58:20.0,01-11-2020 00:04,NaN,NaN,NaN,NaN,103.225.177.108,NaN,NaN,NaN,sujit,NaN,jha,sujit jha,5.024591,0,5,0,3,8,0,[],2,0.0,3


### Registrations with Same Name 

In [36]:
a=df.groupby(['FullName'])['phone_numbers'].count()
NameRep = pd.DataFrame(data=a)
NameRep=NameRep.rename(columns={'phone_numbers':'MultipleSameNameReg'})
df = pd.merge(df,NameRep, how ='left', on ='FullName') 

### DOB (isbday flag)

In [37]:
df['RegisteredOn']= pd.to_datetime(df['RegisteredOn'], dayfirst=True)

In [38]:
df['RegMonth'] = df['RegisteredOn'].dt.month

In [39]:
df['RegDay']= df['RegisteredOn'].astype(str).str[5:10]

In [40]:
df['RegDay']=df['RegDay'].str.replace('-', "")

In [41]:
df['DobCleaned'] = df['Dob'].str.extract('(\d+)')

In [42]:
df['DobMonth'] = df['DobCleaned'].str[:2]
df['DobMonth'] = df['DobMonth'].astype(int)

In [43]:
df['DobDay'] = df['DobCleaned']

In [44]:
#df['DobDay'] = df['DobCleaned'].str[2:]

In [45]:
df['IsBdayFlag(Month)'] = np.where(df['DobMonth']==df['RegMonth'],'flag','')

In [46]:
df['IsBdayFlag(Day)'] = np.where(df['DobDay']==df['RegDay'],'flag','')

### Computing the Total Login Time

In [47]:
df['RegisteredOn']= pd.to_datetime(df['RegisteredOn'], dayfirst=True)

In [48]:
df['LastLoginTime(sec)']=df['LastLoginTime'].str.extract(':(\S\S)')

In [49]:
df['LastLoginTime(hr)'] = df['LastLoginTime'].str.extract('([^.]+$)')
df['LastLoginTime(hr)']=df['LastLoginTime(hr)'].astype(float)


In [50]:
df['LastLoginTime(min)'] =df['LastLoginTime'].str.extract('(\S\S)')
df['LastLoginTime(min)']=df['LastLoginTime(min)'].astype(float)


In [51]:
df['RegisteredOn(Hour)']=df['RegisteredOn'].dt.hour
df['RegisteredOn(Hour)']=df['RegisteredOn(Hour)'].astype(float)
df['RegisteredOn(Min)']=df['RegisteredOn'].dt.minute
df['RegisteredOn(Min)']=df['RegisteredOn(Min)'].astype(float)

In [52]:
df['LoginTimeInHrs'] = df['RegisteredOn(Hour)']-df['LastLoginTime(hr)']
df['LoginTimeInMin'] = df['RegisteredOn(Min)']-df['LastLoginTime(min)']

### Average Login Time

In [53]:
df['AverageLoginTime(Hours)']=df['LoginTimeInHrs'].mean()
df['AverageLoginTime(Min)']=df['LoginTimeInMin'].mean()

In [54]:
#df['LastPinChangeTime'] = pd.to_datetime(df['LastPinChangeTime'])
#df['LastPinChangeTime'].head(20)

In [55]:
#df['AveragePinChangeTime']=df['LastPinChangeTime'].mean()
#df['AveragePinChangeTime'].head()
#cant perform an average on the timestamp

In [56]:
#df['TimeTakenMoreThanAvgPinChangeTime'] = df['AveragePinChangeTime']- df['LastPinChangeTime']
#df['TimeTakenMoreThanAvgPinChangeTime']=df['TimeTakenMoreThanAvgPinChangeTime'].dt.total_seconds()


###  Deviation From Avg Login Time

In [57]:
df['DeviationFromAvgLoginTime(Hrs)'] = df['LoginTimeInHrs']-df['AverageLoginTime(Hours)'] 

In [58]:
df['DeviationFromAvgLoginTime(Min)'] = df['LoginTimeInMin']-df['AverageLoginTime(Min)']

In [59]:
df['DeviationLoginTime-Hr']= np.where(df['DeviationFromAvgLoginTime(Hrs)']>0,'flag','')

In [60]:
df['DeviationLoginTime-Min']= np.where(df['DeviationFromAvgLoginTime(Min)']>0,'flag','')

In [61]:
#DOB not disclosed
#hence we dont compute the age of the user

In [62]:
#now= pd.Timestamp('now')
#df['Dob'] = pd.to_datetime(df['Dob'])
#df['Dob'] = df['Dob'].where(df['Dob'] < now, df['Dob'] -  np.timedelta64(100, 'Y'))
#df['Age'] = (now - df['Dob']).astype('<m8[Y]') 

### RegisteredUsersPerEmailId

In [63]:
new = df.groupby(['Email'])['phone_numbers'].count()
RegisteredUsersPerEmail = pd.DataFrame(data=new)
RegisteredUsersPerEmail=RegisteredUsersPerEmail.rename(columns={'phone_numbers':'UsersPerEmail'})
df = pd.merge(df, RegisteredUsersPerEmail, how ='outer', on ='Email') 

In [64]:
df.head(3)

,FirstName,MiddleName,LastName,Dob,Gender,UserId,Status,Email,phone_numbers,ProgramId,ProgramName,PostalCode,BadTryCount,LastBadLoginTryTime,LastLoginTime,LastPinChangeTime,PresentLoginTime,RegisteredOn,OS Type,Reg Referral Code,Reg Referral Prefix,Ref PC AC Number,ip,Device Id,Sim Id,Meta Data,FirstNameLower,MiddleNameLower,LastNameLower,FullName,Score,IsGibberish,FirstNameLen,MiddleNameLen,LastNameLen,TotalNameLength,EnglishWordsCount,EnglishWords,TotalWordsCount,PctEnglishWordsCount,Vowels,MultipleSameNameReg,RegMonth,RegDay,DobCleaned,DobMonth,DobDay,IsBdayFlag(Month),IsBdayFlag(Day),LastLoginTime(sec),LastLoginTime(hr),LastLoginTime(min),RegisteredOn(Hour),RegisteredOn(Min),LoginTimeInHrs,LoginTimeInMin,AverageLoginTime(Hours),AverageLoginTime(Min),DeviationFromAvgLoginTime(Hrs),DeviationFromAvgLoginTime(Min),DeviationLoginTime-Hr,DeviationLoginTime-Min,UsersPerEmail
0,Krishna,Nagnathrao,Patil,xxxx0908,U,255xx240,1,NaN,9707622140,6019,PayZapp,431717.0,0,NaN,14:44.0,NaN,35:21.0,2020-11-01 00:00:00,Android,NaN,NaN,NaN,49.35.221.2,anid:INE9EIU2dobAh3xITmtgvkw7mck8OVKbztVHLRW2R...,anid:INE9EIU2dobAh3xITmtgvkw7mck8OVKbztVHLRW2R...,7eba668f-d06e-492c-b008-f4b169492b1c,krishna,nagnathrao,patil,krishna nagnathrao patil,4.857123,0,7,10,5,22,0,[],3,0.0,8,1,11,1101,0908,9,0908,,,44,0.0,14.0,0.0,0.0,0.0,-14.0,13.917757,0.249533,-13.917757,-14.249533,,,NaN
1,Jayasri,NaN,Kota,xxxx0913,U,255xx148,1,NaN,9464191867,6019,PayZapp,521001.0,0,41:12.0,42:07.0,41:56.0,59:06.0,2020-11-01 00:01:00,Android,NaN,NaN,NaN,117.197.223.15,anid:ZipdGGO206z2mOdhJ/PkUe6Bzp4fmjZa28MZZA3Rg...,anid:ZipdGGO206z2mOdhJ/PkUe6Bzp4fmjZa28MZZA3Rg...,d1499c8d-cbe9-48ae-b40e-963ca0898b60,jayasri,NaN,kota,jayasri kota,5.010674,0,7,0,4,11,0,[],2,0.0,5,1,11,1101,0913,9,0913,,,07,0.0,42.0,0.0,1.0,0.0,-41.0,13.917757,0.249533,-13.917757,-41.249533,,,NaN
2,rahul,NaN,Verma,xxxx0225,U,255xx696,1,NaN,7074212592,6019,PayZapp,184205.0,0,NaN,59:05.0,NaN,48:31.0,2020-11-01 00:03:00,Android,NaN,NaN,NaN,205.253.177.88,anid:cm0e88QJqfwMxBC7KOKxKdOkT3GirG8maWuBPBFol...,anid:cm0e88QJqfwMxBC7KOKxKdOkT3GirG8maWuBPBFol...,6baa013f-7566-4f8f-9c7c-89d9c5117f84,rahul,NaN,verma,rahul verma,4.695897,0,5,0,5,10,0,[],2,0.0,4,2,11,1101,0225,2,0225,,,05,0.0,59.0,0.0,3.0,0.0,-56.0,13.917757,0.249533,-13.917757,-56.249533,,,NaN


### Email 

### Extraction of Domain name and count on same

In [65]:
df['Domain'] = df['Email'].str.split('@').str[1]

In [66]:
c = df.groupby(['Domain'])['phone_numbers'].count()
c = pd.DataFrame(data=c)
c = c.rename(columns= {'phone_numbers' : 'DomainCount'})
df = pd.merge(df,c, how='outer', on='Domain')

### Special char in email- returns False if there is a special char

In [67]:
df['EmailBody'] = df['Email'].str.split('@').str[0]

In [68]:
df['EmailCheck'] = df['EmailBody'].str.isalnum()
df['EmailCheck']=np.where(df['EmailCheck']=='False','True','False')

### Extracting Special Char and count on the same 

In [69]:
 #extracting all except alphabets,numbers,space,'.','_'
df['SpclChar_InEmail'] = df['EmailBody'].str.extract('([^A-Za-z0-9\s\.\_]+)')
df['SpclChar_InEmail'].value_counts()

+    68
Name: SpclChar_InEmail, dtype: int64

In [70]:
c = df.groupby(['SpclChar_InEmail'])['Email'].count()
c = pd.DataFrame(data=c)
c = c.rename(columns= {'Email' : 'SpclCharCount'})
df = pd.merge(df,c, how='outer', on='SpclChar_InEmail')

In [71]:
df['SpclCharFlag']=np.where(df['SpclCharCount']>0,'flag',0)

### '+' character flag 

In [72]:
df['PlusCharFlag']=np.where(df['SpclChar_InEmail']=='+','flag',' ')

In [73]:
df['PlusCharFlag'].value_counts()

        1417
flag      68
Name: PlusCharFlag, dtype: int64

### Number Char in Email & count on the same

In [74]:
df['NumChar_InEmail']=df['EmailBody'].str.replace(r'\D+','')

In [75]:
c=df.groupby(['NumChar_InEmail'])['phone_numbers'].count()
c = pd.DataFrame(data=c)
c = c.rename(columns= {'phone_numbers' : 'NumCharCount'})
df = pd.merge(df,c, how='outer', on='NumChar_InEmail')

In [76]:
df['NumCharCount'].value_counts()

1.0     175
65.0     65
2.0      26
3.0       6
4.0       4
Name: NumCharCount, dtype: int64

In [77]:
df['NumCharFlag']=np.where(df['NumCharCount']<10, 'flag','')

### Email text & count on the same 

In [78]:
df['Email_text'] = df['Email'].str.extract('([a-zA-Z ]+)', expand=False).str.strip()

In [79]:
c=df.groupby(['Email_text'])['Email'].count()
c = pd.DataFrame(data=c)
c = c.rename(columns= {'Email' : 'EmailTextCount'})
df = pd.merge(df,c, how='outer', on='Email_text')

In [80]:
df['EmailTextFlag']=np.where(df['EmailTextCount']>1, 'flag','')

In [81]:
#const input = 12345
#const pattern = '0123456789012345789'
#if (~pattern.indexOf(input))
#  console.log('pattern in input')
#else
#  console.log('pattern not in input')

## Postal Code

### Users Per Postal Code

In [82]:
c = df.groupby(['PostalCode'])['phone_numbers'].count()
UsersPerPostalCode = pd.DataFrame(data=c)
UsersPerPostalCode = UsersPerPostalCode.rename(columns= {'phone_numbers' : 'PostalCodeCounts'})
df = pd.merge(df, UsersPerPostalCode, how='outer', on='PostalCode')


In [83]:
#!pip install geopy
#from geopy.geocoders import Nominatim


In [84]:
# Importing required module 

#def geodata(code):
    # Using Nominatim Api 
    #geolocator = Nominatim(user_agent="geoapiExercises") 
  
 # Zipocde input 
#zipcode = "560100"
  
# Using geocode() 
    #location = geolocator.geocode(code) 
 # Displaying address details 
    
   # return location


In [85]:
#df['pincodederivedloc']=df['PostalCode'][:100].apply(geodata)

In [86]:
#df["pincodederivedloc"].fillna("Invalid", inplace = True) 

### Pincode Matching and deriving Location from the same

In [87]:
df_pinreg = pd.read_csv("pincoderegistry.csv")

In [88]:
df_pinreg['PostalCode']=df_pinreg['PostalCode'].astype(object)

In [89]:
df_pinreg = df_pinreg.rename(columns={'regionname':'RegionName_govdata'})
df_pinreg = df_pinreg.rename(columns={'circlename':'CircleName_govdata'})
df_pinreg = df_pinreg.rename(columns={'taluk':'Taluk_govdata'})
df_pinreg = df_pinreg.rename(columns={'districtname':'DistrictName_govdata'})
df_pinreg = df_pinreg.rename(columns={'statename':'StateName_govdata'})

In [90]:
df['PostalCode'].isnull().sum()

24

In [91]:
n = df.PostalCode.isin(df_pinreg.PostalCode)
n.value_counts()

True     1316
False     169
Name: PostalCode, dtype: int64

In [92]:
df['PostalCodeCheck'] = np.where(df['PostalCode'].isnull() ,' ',n)

In [93]:
df['PostalCodeCheck'].value_counts()

True     1316
False     145
           24
Name: PostalCodeCheck, dtype: int64

In [94]:
df = df.merge(df_pinreg[['PostalCode',
       'RegionName_govdata', 'CircleName_govdata', 'Taluk_govdata', 'DistrictName_govdata', 'StateName_govdata']] , on='PostalCode',how='left')

###  PostalCode counts per Referral Code

In [95]:
r = df.groupby(['Reg Referral Code'])['PostalCode'].count()
PostalsPerRef = pd.DataFrame(data=r)
PostalsPerRef = PostalsPerRef.rename(columns= {'PostalCode' : 'PostalCountsPerRef'})
df = pd.merge(df, PostalsPerRef, how='outer', on='Reg Referral Code')

## Registrations per A/C number

In [96]:
m = df.groupby(['Ref PC AC Number'])['phone_numbers'].count()
UsersPerAC = pd.DataFrame(data=m)
UsersPerAC = UsersPerAC.rename(columns= {'phone_numbers' : 'UsersPerAccount'})
df = pd.merge(df, UsersPerAC, how='outer', on='Ref PC AC Number')


## IP 

### Registered User Count Per IP Address

In [97]:
d = df.groupby(['ip'])['phone_numbers'].count()
UsersPerIP = pd.DataFrame(data=d)
UsersPerIP = UsersPerIP.rename(columns= {'phone_numbers' : 'UserCountsPerIP'})
df = pd.merge(df, UsersPerIP, how='outer', on='ip')


### Users per IP Range
### Users Per IP Block

In [98]:
df['IPBlock1'] = df['ip'].str.extract('(\d+\.\d+\.\d+)')
df['IPBlock1']

0         49.35.221
1       117.197.223
2       205.253.177
3       103.225.177
4        157.35.224
           ...     
1480    106.213.183
1481      49.36.230
1482    103.242.123
1483      49.206.44
1484    103.206.136
Name: IPBlock1, Length: 1485, dtype: object

In [99]:
f = df.groupby(['IPBlock1'])['phone_numbers'].count()
UsersPerBlock1 = pd.DataFrame(data=f)
UsersPerBlock1 = UsersPerBlock1.rename(columns= {'phone_numbers' : 'UsersPerBlock1'})
df = pd.merge(df, UsersPerBlock1, how='outer', on='IPBlock1')

In [100]:
df['IPBlock2'] = df['ip'].str.extract('(\d+\.\d+)')

In [101]:
z = df.groupby(['IPBlock2'])['phone_numbers'].count()
UsersPerBlock2 = pd.DataFrame(data=z)
UsersPerBlock2 = UsersPerBlock2.rename(columns= {'phone_numbers' : 'UsersPerBlock2'})
df = pd.merge(df, UsersPerBlock2, how='outer', on='IPBlock2')

In [102]:
df['IPBlock3'] = df['ip'].str.extract('(\d+)')

In [103]:
y = df.groupby(['IPBlock3'])['phone_numbers'].count()
UsersPerBlock3 = pd.DataFrame(data=y)
UsersPerBlock3 = UsersPerBlock3.rename(columns= {'phone_numbers' : 'UsersPerBlock3'})
df = pd.merge(df, UsersPerBlock3, how='outer', on='IPBlock3')

## Device 

### Registrations per Device Id

#### Filter for the blanks 

In [104]:
df.shape

(1485, 93)

In [105]:
#dfd=df[df['Device Id'].notnull()]
#dfd.shape

In [106]:
#dfd[' Meta Data'].isnull().sum()

In [107]:
e = df.groupby(['Device Id'])['phone_numbers'].count()
UsersPerDeviceId = pd.DataFrame(data=e)
UsersPerDeviceId = UsersPerDeviceId.rename(columns= {'phone_numbers' : 'UserCountPerDeviceId'})
df = pd.merge(df, UsersPerDeviceId, how='outer', on='Device Id')

### Registrations per Sim Id

In [108]:
f = df.groupby(['Sim Id'])['phone_numbers'].count()
UsersPerSimId = pd.DataFrame(data=f)
UsersPerSimId = UsersPerSimId.rename(columns= {'phone_numbers' : 'UserCountPerSimId'})
df = pd.merge(df, UsersPerSimId, how='outer', on='Sim Id')

### Meta Data Counts

In [109]:
#dfm=df[df[' Meta Data'].notnull()]
#dfm.shape

In [110]:
#dfm[' Meta Data'].isnull().sum()

In [111]:
o = df.groupby([' Meta Data'])['phone_numbers'].count()
RegisteredUsersPerMetaData = pd.DataFrame(data=o)
RegisteredUsersPerMetaData =RegisteredUsersPerMetaData .rename(columns={'phone_numbers':'UsersPerMetaData'})
df = pd.merge(df, RegisteredUsersPerMetaData, how ='outer', on =' Meta Data') 

### Referral Code Velocity - frequency usage of each referral code

In [112]:
# drop for the hourly and 2 hourly buckets for now

In [113]:
#z = df['Reg Referral Code'].value_counts()

In [114]:
#z1 = z.to_dict() #converts to dictionary
#df['RefCodesVelocity'] = df['Reg Referral Code'].map(z1) 

In [115]:
e=df.groupby(['Reg Referral Code'])['phone_numbers'].count()
e = pd.DataFrame(data=e)
e = e.rename(columns={'phone_numbers':'RefCodeVelocity'})
df = pd.merge(df,e,how='outer',on='Reg Referral Code')

## Deriving Location Details from IP address and Mobile

In [116]:
dfm = pd.read_csv('PhoneNumbers_With_location.csv')

In [117]:
dfm = pd.read_csv('PhoneNumbers_With_location.csv')
dfm['phonenumber']=dfm['phonenumber'].astype(str)
dfm['phonenumber']=dfm['phonenumber'].str[2:]

In [118]:
dfm = dfm.rename(columns={'phonenumber':'Mobile_API'})
dfm = dfm.rename(columns={'is_valid':'IsValidMobile_API'})
dfm = dfm.rename(columns={'local_format':'LocalFormatMobile_API'})
dfm = dfm.rename(columns={'international_format':'IntFormatMobile_API'})
dfm = dfm.rename(columns={'country_prefix':'CountryPrefixMobile_API'})
dfm = dfm.rename(columns={'country_code':'CountryCodeMobile_API'})
dfm = dfm.rename(columns={'location':'LocationMobile_API'})
dfm = dfm.rename(columns={'carrier':'CarrierMobile_API'})
dfm = dfm.rename(columns={'line_type':'LineTypeMobile_API'})
dfm=dfm.drop(['json'],axis=1)

In [119]:
dfi = pd.read_csv('IPs_With_location.csv')
dfi.shape

(1485, 9)

In [120]:
dfi = dfi.rename(columns={'country_code':'CountryCodeIP_API'})
dfi = dfi.rename(columns={'country_name':'CountryNameIP_API'})
dfi = dfi.rename(columns={'region_name':'RegionNameIP_API'})
dfi = dfi.rename(columns={'city_name':'CityNameIP_API'})
dfi = dfi.rename(columns={'zip_code':'ZipcodeIP_API'})
dfi=dfi.drop(['json'],axis=1)

In [121]:
f3 = df.merge(dfi.drop_duplicates(subset=['ip']), how='left', on='ip')

In [122]:
f3['phone_numbers']=f3['phone_numbers'].astype(str)
dfm['Mobile_API']=dfm['Mobile_API'].astype(str)

In [123]:
f3=pd.merge(f3,dfm, how='left', left_on='phone_numbers', right_on='Mobile_API')

In [124]:
f3.shape

(1485, 114)

# Columns further Derived from the 1st version of output

### Matching of Country (IPaddress)

In [125]:
f3['CountryNameIP_API'] = f3['CountryNameIP_API'].replace(np.nan," ",regex=True)

In [126]:
conditions  = [(f3['CountryNameIP_API'] == "India"),(f3['CountryNameIP_API']==' ')]
choices     = [('Valid'),(' ')]
f3['CountryMatching(IP)'] = np.select(conditions, choices, default='Invalid')

### Matching of Country Code(IP Address)

In [127]:
f3['CountryCodeIP_API']=f3['CountryCodeIP_API'].replace(np.nan," ",regex=True)

In [128]:
conditions  = [(f3['CountryCodeIP_API'] == "IN"),(f3['CountryCodeIP_API'] ==' ')]
choices     = [('Valid'),(' ')]
f3['CountryCodeMatching(IP)'] = np.select(conditions, choices, default='Invalid')

In [129]:
f3['CountryCodeMatching(IP)'].value_counts()


Valid      1000
            483
Invalid       2
Name: CountryCodeMatching(IP), dtype: int64

### Matching of Country Code(Mobile)

In [130]:
f3['CountryCodeMobile_API'] =f3['CountryCodeMobile_API'].replace(np.nan," ",regex=True)

In [131]:
conditions  = [(f3['CountryCodeMobile_API'] == "IN"),(f3['CountryCodeMobile_API'] ==' ')]
choices     = [('Valid'),(' ')]
f3['CountryCodeMatching(Mobile)'] = np.select(conditions, choices, default='Invalid')

### Matching of State through Pincode

In [132]:
f3['PostalCode'] = f3['PostalCode'].astype(str)
f3['ExtractedPinDigits(State)'] = f3['PostalCode'].str.slice(0, 1) 

In [133]:
f3['ZipcodeIP_API'] = f3['ZipcodeIP_API'].astype(str)
f3['ExtractedPinDigits(IP-State)'] = f3['ZipcodeIP_API'].str.slice(0, 1) 

In [134]:
f3['PostalCodeCheck'] = f3['PostalCodeCheck'].astype(str)

In [135]:
conditions  = [(f3['PostalCodeCheck']==' '),(f3['PostalCodeCheck']=='False'),(f3['ZipcodeIP_API']=='nan'),(f3['ExtractedPinDigits(State)'] == f3['ExtractedPinDigits(IP-State)'])]
choices     = [(' '),('Invalid'),(' '),('Match')]
f3['StateMatching'] = np.select(conditions, choices, default='Mismatch')

### Matching of Locality through Pincode

In [136]:
f3['PostalCode'] = f3['PostalCode'].astype(str)
f3['ExtractedPinDigits(Locality)'] = f3['PostalCode'].str.slice(0, 3) 

In [137]:
f3['ZipcodeIP_API'] = f3['ZipcodeIP_API'].astype(str)
f3['ExtractedPinDigits(IP-Locality)'] = f3['ZipcodeIP_API'].str.slice(0, 3) 

In [138]:
conditions  = [(f3['PostalCodeCheck']==' '),(f3['PostalCodeCheck']=='False'),(f3['ZipcodeIP_API']=='nan'),(f3['ExtractedPinDigits(Locality)'] == f3['ExtractedPinDigits(IP-Locality)'])]
choices     = [(' '),('Invalid'),(' '),('Match')]
f3['LocalityMatching'] = np.select(conditions, choices, default='Mismatch')

### Pincode matching with that of IP address

In [139]:
f3['ZipcodeIP_API'].value_counts()

nan         483
400099.0     85
560018.0     57
500018.0     46
412415.0     43
           ... 
796181.0      1
524308.0      1
476111.0      1
574508.0      1
246763.0      1
Name: ZipcodeIP_API, Length: 276, dtype: int64

In [140]:
conditions  = [(f3['PostalCodeCheck']==' '),(f3['ZipcodeIP_API']=='nan'),(f3['PostalCode'] == f3['ZipcodeIP_API'])]
choices     = [(' '),(' '),('Match')]
f3['ZipcodeMatch-UserEntered&IP'] = np.select(conditions, choices, default='Mismatch')

In [141]:
f3['ZipcodeMatch-UserEntered&IP'].value_counts()

Mismatch    974
            505
Match         6
Name: ZipcodeMatch-UserEntered&IP, dtype: int64

In [142]:
f3 = f3.rename(columns={'phone_numbers':'Mobile'})
f3 = f3.rename(columns={'ip':'Reg Client IP'})
f3 = f3.drop(['number'],axis=1)

In [143]:
f3.to_csv('RegSignal.csv', index=False)

In [144]:
print(f3.columns.tolist())

['FirstName', 'MiddleName', 'LastName', 'Dob', 'Gender', 'UserId', 'Status', 'Email', 'Mobile', 'ProgramId', 'ProgramName', 'PostalCode', 'BadTryCount', 'LastBadLoginTryTime', 'LastLoginTime', 'LastPinChangeTime', 'PresentLoginTime', 'RegisteredOn', 'OS Type', 'Reg Referral Code', 'Reg Referral Prefix', 'Ref PC AC Number', 'Reg Client IP', 'Device Id', 'Sim Id', ' Meta Data', 'FirstNameLower', 'MiddleNameLower', 'LastNameLower', 'FullName', 'Score', 'IsGibberish', 'FirstNameLen', 'MiddleNameLen', 'LastNameLen', 'TotalNameLength', 'EnglishWordsCount', 'EnglishWords', 'TotalWordsCount', 'PctEnglishWordsCount', 'Vowels', 'MultipleSameNameReg', 'RegMonth', 'RegDay', 'DobCleaned', 'DobMonth', 'DobDay', 'IsBdayFlag(Month)', 'IsBdayFlag(Day)', 'LastLoginTime(sec)', 'LastLoginTime(hr)', 'LastLoginTime(min)', 'RegisteredOn(Hour)', 'RegisteredOn(Min)', 'LoginTimeInHrs', 'LoginTimeInMin', 'AverageLoginTime(Hours)', 'AverageLoginTime(Min)', 'DeviationFromAvgLoginTime(Hrs)', 'DeviationFromAvgLoginT

In [145]:
f3.shape

(1485, 123)

In [146]:
f3['CountryMatching(IP)'].value_counts()

Valid      1000
            483
Invalid       2
Name: CountryMatching(IP), dtype: int64

In [147]:
f3['CountryCodeMatching(IP)'].value_counts()

Valid      1000
            483
Invalid       2
Name: CountryCodeMatching(IP), dtype: int64

In [148]:
f3['CountryCodeMatching(Mobile)'].value_counts()

         1448
Valid      37
Name: CountryCodeMatching(Mobile), dtype: int64

In [149]:
f3['StateMatching'].value_counts()

Match       812
            384
Invalid     145
Mismatch    144
Name: StateMatching, dtype: int64

In [150]:
f3['LocalityMatching'].value_counts()

Mismatch    731
            384
Match       225
Invalid     145
Name: LocalityMatching, dtype: int64

In [151]:
f3['ZipcodeMatch-UserEntered&IP'].value_counts()

Mismatch    974
            505
Match         6
Name: ZipcodeMatch-UserEntered&IP, dtype: int64

# Creating an Index to classify records as Abuse and Non-Abuse

## Sub-indexing

In [152]:
df = pd.read_csv('RegSignal.csv')

In [153]:
df['ScoreNorm']=(df['Score']-df['Score'].min())/(df['Score'].max()-df['Score'].min())
df['ScoreNorm'] = df['ScoreNorm'].apply(lambda x: x*100)
df['ScoreNormRev'] = df['ScoreNorm'].apply(lambda x: 100-x)
df['GibberishFlagNorm']=(df['IsGibberish']-df['IsGibberish'].min())/(df['IsGibberish'].max()-df['IsGibberish'].min())
df['GibberishFlagNorm'] = df['GibberishFlagNorm'].apply(lambda x: x*100)
#from scipy import stats
#df['ScoreNorm3'] = np.abs(stats.zscore(df["Score"]))


# # Device
# In[15]:
#MEAN FOR NON-BLANK ROWS
df['UsersPerMetaData ']=df['UsersPerMetaData'].fillna(df['UsersPerMetaData'].mean())
df['UsersPerMetaDataNorm']= (df['UsersPerMetaData ']-df['UsersPerMetaData '].min())/(df['UsersPerMetaData '].max()-df['UsersPerMetaData '].min())
df['UsersPerMetaDataNorm'] = df['UsersPerMetaDataNorm'].apply(lambda x: x*100)
df['UserCountPerDeviceId ']=df['UserCountPerDeviceId'].fillna(df['UserCountPerDeviceId'].mean())
df['UserCountPerDeviceIdNorm']= (df['UserCountPerDeviceId ']-df['UserCountPerDeviceId '].min())/(df['UserCountPerDeviceId '].max()-df['UserCountPerDeviceId '].min())
df['UserCountPerDeviceIdNorm'] = df['UserCountPerDeviceIdNorm'].apply(lambda x: x*100)
df['UserCountPerSimId ']=df['UserCountPerSimId'].fillna(df['UserCountPerSimId'].mean())
#df['UserCountPerSimIdNorm']= (df['UserCountPerSimId ']-df['UserCountPerSimId '].min())/(df['UserCountPerSimId '].max()-df['UserCountPerSimId '].min())
df['UserCountPerSimIdNorm'] = df['UserCountPerSimId '].apply(lambda x: x*100)


# # Zipcode
#df['PostalCodeCounts'] = df['PostalCodeCounts'].fillna(df['PostalCodeCounts'].mean())
#df['PostalCodeCountsNorm']= (df['PostalCodeCounts']-df['PostalCodeCounts'].min())/(df['PostalCodeCounts'].max()-df['PostalCodeCounts'].min())
#df['PostalCodeCountsNorm'] = df['PostalCodeCountsNorm'].apply(lambda x: x*100)
conditions  = [(df['PostalCodeCheck']== 'True'),(df['PostalCodeCheck']== 'False')]
choices     = [0,1]
df['PostalCodeCheck '] = np.select(conditions, choices, default=0.097)
df['PostalCodeCheckNorm']= (df['PostalCodeCheck ']-df['PostalCodeCheck '].min())/(df['PostalCodeCheck '].max()-df['PostalCodeCheck '].min())
df['PostalCodeCheckNorm'] = df['PostalCodeCheckNorm'].apply(lambda x: x*100)


#IP
df['UserCountsPerIPNorm']=(df['UserCountsPerIP']-df['UserCountsPerIP'].min())/(df['UserCountsPerIP'].max()-df['UserCountsPerIP'].min())
df['UserCountsPerIPNorm']=df['UserCountsPerIPNorm'].apply(lambda x: x*100)
# In[8]:
df['UsersPerBlock1Norm']=(df['UsersPerBlock1']-df['UsersPerBlock1'].min())/(df['UsersPerBlock1'].max()-df['UsersPerBlock1'].min())
df['UsersPerBlock1Norm']=df['UsersPerBlock1Norm'].apply(lambda x: x*100)
# In[9]:
df['UsersPerBlock2Norm']=(df['UsersPerBlock2']-df['UsersPerBlock2'].min())/(df['UsersPerBlock2'].max()-df['UsersPerBlock2'].min())
df['UsersPerBlock2Norm']=df['UsersPerBlock2Norm'].apply(lambda x: x*100)
# In[10]:
df['UsersPerBlock3Norm']=(df['UsersPerBlock3']-df['UsersPerBlock3'].min())/(df['UsersPerBlock3'].max()-df['UsersPerBlock3'].min())
df['UsersPerBlock3Norm']=df['UsersPerBlock3Norm'].apply(lambda x: x*100)


# # Referral Code
df['RefCodesVelocity']=df['RefCodeVelocity'].fillna(df['RefCodeVelocity'].mean())
df['RefCodesVelocityNorm'] = (df['RefCodesVelocity']-df['RefCodesVelocity'].min())/(df['RefCodesVelocity'].max()-df['RefCodesVelocity'].min())
df['RefCodesVelocityNorm']=df['RefCodesVelocityNorm'].apply(lambda x: x*100)


# # Email
#fillna with average for non-null email
df['SpclCharFlag'] = np.where(df['SpclCharFlag']=='flag',1,0)
conditions  = [(df['Email'].isnull()),(df['Email'].notnull())]
choices     = [(df['SpclCharCount'].fillna(df['SpclCharFlag'].mean())),(df['SpclCharCount'].fillna(df['SpclCharFlag'].min()))]
df["SpclCharCount "] = np.select(conditions, choices, default='')
df['SpclCharCount '] = df['SpclCharCount '].astype(float)
df['SpclCharCountNorm'] = (df['SpclCharCount ']-df['SpclCharCount '].min())/(df['SpclCharCount '].max()-df['SpclCharCount '].min())
df['SpclCharCountNorm']=df['SpclCharCount '].apply(lambda x: x*100)

df['PlusCharFlag']= np.where(df['PlusCharFlag']=='flag',1,0)

conditions  = [(df['Email'].isnull()),(df['PlusCharFlag']==1)]
choices     = [(df['PlusCharFlag'].mean()),(1) ]
df["PlusCharFlag "] = np.select(conditions, choices, default=0)


conditions  = [(df['Email'].isnull()),(df['Email'].notnull())]
choices     = [(df['NumCharCount'].fillna(df['NumCharCount'].mean())),(df['NumCharCount'].fillna(df['NumCharCount'].max()))]
df["NumCharCount "] = np.select(conditions, choices, default='')
df['NumCharCount '] = df['NumCharCount '].astype(float)
df['NumCharCountNorm'] = (df['NumCharCount ']-df['NumCharCount '].min())/(df['NumCharCount '].max()-df['NumCharCount '].min())
df['NumCharCountNorm']=df['NumCharCountNorm'].apply(lambda x: x*100)


conditions  = [(df['Email'].isnull()),(df['Email'].notnull())]
choices     = [(df['EmailTextCount'].fillna(df['EmailTextCount'].mean())),(df['EmailTextCount'].fillna(df['EmailTextCount'].min()))]
df["EmailTextCount "] = np.select(conditions, choices, default='')
df['EmailTextCount '] = df['EmailTextCount '].astype(float)
df['EmailTextCountNorm'] = (df['EmailTextCount ']-df['EmailTextCount '].min())/(df['EmailTextCount '].max()-df['EmailTextCount '].min())
df['EmailTextCountNorm']=df['EmailTextCountNorm'].apply(lambda x: x*100)


In [154]:
conditions  = [(df['CountryMatching(IP)'] == "Valid"),(df['CountryMatching(IP)']=='Invalid')]
choices     = [0,1]
df['CountryMatching'] = np.select(conditions, choices, default=0.001)

In [155]:
#df['CountryMatchingNorm'] = (df['CountryMatching']-df['CountryMatching'].min())/(df['CountryMatching'].max()-df['CountryMatching'].min())
df['CountryMatchingNorm']=df['CountryMatching'].apply(lambda x: x*100)

In [156]:
conditions  = [(df['StateMatching'] == "Match"),(df['StateMatching']=='Mismatch'),(df['StateMatching']=='Invalid')]
choices     = [0,1,1]
df['StateMatching '] = np.select(conditions, choices, default=0.21)

In [157]:
#df['StateMatchingNorm'] = (df['StateMatching ']-df['StateMatching '].min())/(df['StateMatching '].max()-df['StateMatching '].min())
df['StateMatchingNorm']=df['StateMatching '].apply(lambda x: x*100)

In [158]:
conditions  = [(df['LocalityMatching'] == "Match"),(df['LocalityMatching']=='Mismatch'),(df['LocalityMatching']=='Invalid')]
choices     = [0,1,1]
df['LocalityMatching '] = np.select(conditions, choices, default=0.605)

In [159]:
#df['LocalityMatchingNorm'] = (df['LocalityMatching ']-df['LocalityMatching '].min())/(df['LocalityMatching '].max()-df['LocalityMatching '].min())
df['LocalityMatchingNorm'] = df['LocalityMatching '].apply(lambda x: x*100)

In [160]:
conditions  = [(df['ZipcodeMatch-UserEntered&IP'] == "Match"),(df['ZipcodeMatch-UserEntered&IP']=='Mismatch')]
choices     = [0,1]
df['ZipcodeMatch-UserEntered&IP '] = np.select(conditions, choices, default=0.67)

In [161]:
#df['ZipcodeMatch-UserEntered&IPNorm'] = (df['ZipcodeMatch-UserEntered&IP ']-df['ZipcodeMatch-UserEntered&IP '].min())/(df['ZipcodeMatch-UserEntered&IP '].max()-df['ZipcodeMatch-UserEntered&IP '].min())
df['ZipcodeMatch-UserEntered&IPNorm']=df['ZipcodeMatch-UserEntered&IP '].apply(lambda x: x*100)

In [162]:
conditions  = [(df['CountryCodeMatching(Mobile)'] == "Valid"),(df['CountryCodeMatching(Mobile)']=='Invalid')]
choices     = [0,1]
df['CountryCodeMatching(Mobile) '] = np.select(conditions, choices, default=0.001)

In [163]:
#df['CountryCodeMatching(Mobile)Norm'] = (df['CountryCodeMatching(Mobile) ']-df['CountryCodeMatching(Mobile) '].min())/(df['CountryCodeMatching(Mobile) '].max()-df['CountryCodeMatching(Mobile) '].min())
df['CountryCodeMatching(Mobile)Norm']=df['CountryCodeMatching(Mobile) '].apply(lambda x: x*100)

In [164]:
# # Registration Date/Time
df['IsBdayFlag(Month)'] = np.where(df['IsBdayFlag(Month)']=='flag',1,0)
#df['IsBdayFlag(Month)Norm'] = (df['IsBdayFlag(Month)']-df['IsBdayFlag(Month)'].min())/(df['IsBdayFlag(Month)'].max()-df['IsBdayFlag(Month)'].min())
df['IsBdayFlag(Month)Norm']=df['IsBdayFlag(Month)'].apply(lambda x: x*100)

# In[22]:
df['IsBdayFlag(Day)'] = np.where(df['IsBdayFlag(Day)']=='flag',1,0)
#df['IsBdayFlag(Day)Norm'] = (df['IsBdayFlag(Day)']-df['IsBdayFlag(Day)'].min())/(df['IsBdayFlag(Day)'].max()-df['IsBdayFlag(Day)'].min())
df['IsBdayFlag(Day)Norm']=df['IsBdayFlag(Day)'].apply(lambda x: x*100)
df['DeviationLoginTime-Hr'] = np.where(df['DeviationLoginTime-Hr']=='flag',1,0)
df['DeviationLoginTime-HrNorm'] = (df['DeviationLoginTime-Hr']-df['DeviationLoginTime-Hr'].min())/(df['DeviationLoginTime-Hr'].max()-df['DeviationLoginTime-Hr'].min())
df['DeviationLoginTime-HrNorm']=df['DeviationLoginTime-HrNorm'].apply(lambda x: x*100)
df['DeviationLoginTime-Min'] = np.where(df['DeviationLoginTime-Min']=='flag',1,0)
df['DeviationLoginTime-MinNorm'] = (df['DeviationLoginTime-Min']-df['DeviationLoginTime-Min'].min())/(df['DeviationLoginTime-Min'].max()-df['DeviationLoginTime-Min'].min())
df['DeviationLoginTime-MinNorm']=df['DeviationLoginTime-MinNorm'].apply(lambda x: x*100)



## High-level Indexing

In [165]:
score= []
for value in df['MultipleSameNameReg']:
    if value<=2:
        score.append(1)
    elif value>2 and value<=5:
        score.append(10)
    elif value>5 and value<=10:
        score.append(20)
    elif value>10 and value<=20:
        score.append(30)
    elif value>20 and value<=50:
        score.append(50)
    elif value>50 and value<=70:
        score.append(70)
    elif value>70 and value<=100:
        score.append(90)
    else:
        score.append(100)
df['MultipleRegistrations']= score        

In [166]:
df['MultipleRegistrations'].value_counts()

1     1455
30      20
10      10
Name: MultipleRegistrations, dtype: int64

In [167]:
# # Name
df['WtdScore'] =  df['ScoreNorm'].apply(lambda x: x*0.60)
df['WtdGibFlag']= df['GibberishFlagNorm'].apply(lambda x: x*0.40)
df['Name'] = df['WtdScore']+df['WtdGibFlag']
df['Name'] = (df['Name']-df['Name'].min())/(df['Name'].max()-df['Name'].min())
df['Name'] = df['Name'].apply(lambda x: x*100)


# # Zipcode

# In[67]:


df['WtdPostalCodeCheckNorm'] = df['PostalCodeCheckNorm'].apply(lambda x: x*0.8)
#df['WtdPostalCountsPerRefNorm'] = df['PostalCountsPerRefNorm'].apply(lambda x: x*0.2)
df['Zipcode'] = df['WtdPostalCodeCheckNorm']
df['Zipcode'] = (df['Zipcode']-df['Zipcode'].min())/(df['Zipcode'].max()-df['Zipcode'].min())
df['Zipcode'] = df['Zipcode'].apply(lambda x: x*100)


# # Device

# In[68]:


df['WtdUsersPerMetaDataNorm'] = df['UsersPerMetaDataNorm'].apply(lambda x: x*0.4)
df['WtdUserCountPerSimIdNorm'] = df['UserCountPerSimId '].apply(lambda x: x*0.2)
df['WtdUserCountPerDeviceIdNorm'] = df['UserCountPerDeviceIdNorm'].apply(lambda x: x*0.4)
df['Device'] = df['WtdUsersPerMetaDataNorm']+df['WtdUserCountPerDeviceIdNorm']+df['WtdUserCountPerSimIdNorm']
df['Device'] = (df['Device']-df['Device'].min())/(df['Device'].max()-df['Device'].min())
df['Device'] = df['Device'].apply(lambda x: x*100)


# # Reg Date/Time

df['WtdIsBdayFlag(Month)Norm'] = df['IsBdayFlag(Month)Norm'].apply(lambda x: x*0.18)
df['WtdIsBdayFlag(Day)Norm'] = df['IsBdayFlag(Day)Norm'].apply(lambda x: x*0.35)
df['WtdDeviationLoginTime-HrNorm']=df['DeviationLoginTime-HrNorm'].apply(lambda x: x*0.02)
df['WtdDeviationLoginTime-MinNorm']=df['DeviationLoginTime-MinNorm'].apply(lambda x: x*0.02)
df['Reg Date/Time'] = df['WtdIsBdayFlag(Month)Norm'] + df['WtdIsBdayFlag(Day)Norm'] + df['WtdDeviationLoginTime-HrNorm'] + df['WtdDeviationLoginTime-MinNorm']
df['Reg Date/Time'] = (df['Reg Date/Time']-df['Reg Date/Time'].min())/(df['Reg Date/Time'].max()-df['Reg Date/Time'].min())
df['Reg Date/Time'] = df['Reg Date/Time'].apply(lambda x: x*100)


# # IP

# In[65]:


df['WtdUserCountsPerIPNorm']=df['UserCountsPerIPNorm'].apply(lambda x: x*0.7)
df['WtdUsersPerBlock1Norm']=df['UsersPerBlock1Norm'].apply(lambda x: x*0.2)
df['WtdUsersPerBlock2Norm']=df['UsersPerBlock2Norm'].apply(lambda x: x*0.1)
df['WtdUsersPerBlock3Norm']=df['UsersPerBlock3Norm'].apply(lambda x: x*0.03)
df['WtdCountryMatchingNorm']=df['CountryMatchingNorm'].apply(lambda x: x*0.03)
df['WtdStateMatchingNorm']=df['StateMatchingNorm'].apply(lambda x: x*0.02)
df['WtdLocalityMatchingNorm']=df['LocalityMatchingNorm'].apply(lambda x: x*0.01)
df['WtdZipcodeMatch-UserEntered&IPNorm']=df['ZipcodeMatch-UserEntered&IPNorm'].apply(lambda x: x*0.05)
df['WtdCountryCodeMatching(Mobile)Norm']=df['CountryCodeMatching(Mobile)Norm'].apply(lambda x: x*0.02)

# with the api related features 
df['IP&Phone']=df['WtdUserCountsPerIPNorm']+df['WtdUsersPerBlock1Norm']+df['WtdUsersPerBlock2Norm']+df['WtdUsersPerBlock3Norm']+df['WtdCountryMatchingNorm']+df['WtdStateMatchingNorm']+df['WtdLocalityMatchingNorm']+df['WtdZipcodeMatch-UserEntered&IPNorm']+df['WtdCountryCodeMatching(Mobile)Norm']
df['IP&Phone'] = (df['IP&Phone']-df['IP&Phone'].min())/(df['IP&Phone'].max()-df['IP&Phone'].min())
df['IP&Phone'] = df['IP&Phone'].apply(lambda x: x*100)


# # Referral Codes
df['WtdRefCodesVelocityNorm'] = df['RefCodesVelocityNorm'].apply(lambda x: x*1.0)
df['ReferralCode'] = df['WtdRefCodesVelocityNorm']
df['ReferralCode'] = (df['ReferralCode']-df['ReferralCode'].min())/(df['ReferralCode'].max()-df['ReferralCode'].min())
df['ReferralCode'] = df['ReferralCode'].apply(lambda x: x*100)


# # Email

df['SpclCharCountNorm']=df['SpclCharCountNorm'].apply(lambda x: x*0.25)
df["PlusCharFlagNorm"]=df["PlusCharFlag "].apply(lambda x: x*75)
df['NumCharCountNorm'] = df['NumCharCountNorm'].apply(lambda x: x*0.15)
df['EmailTextCountNorm'] = df['EmailTextCountNorm'].apply(lambda x: x*0.25)
df['Email '] = df['EmailTextCountNorm']+df['NumCharCountNorm']+df['SpclCharCountNorm']+df['PlusCharFlagNorm']
df['Email '] = (df['Email ']-df['Email '].min())/(df['Email '].max()-df['Email '].min())
df['Email '] = df['Email '].apply(lambda x: x*100)

In [168]:
df['Name'] = df['Name'].round(2)
df['Device'] = df['Device'].round(2)
df['Zipcode'] = df['Zipcode'].round(2)
df['IP&Phone'] = df['IP&Phone'].round(2)
df['Reg Date/Time'] = df['Reg Date/Time'].round(2)
df['ReferralCode'] = df['ReferralCode'].round(2)
df['Email '] = df['Email '].round(2)

In [169]:
df['wtdName'] = df['Name'].apply(lambda x: x*0.3)
df['wtdMultipleReg']=df['MultipleRegistrations'].apply(lambda x: x*0.28)
df['wtdZipcode'] =  df['Zipcode'].apply(lambda x: x*0.15)
df['wtdDevice'] =  df['Device'].apply(lambda x: x*0.2)
df['wtdIP&Phone']= df['IP&Phone'].apply(lambda x: x*0.12)
df['wtdReg Date/Time'] =  df['Reg Date/Time'].apply(lambda x: x*0.25)
df['wtdReferralCode'] =  df['ReferralCode'].apply(lambda x: x*0.3)
df['wtdEmail']= df['Email '].apply(lambda x: x*0.1)

# In[33]:


df['Registration_Abuse_Score'] = df['wtdMultipleReg']+df['wtdName']+df['wtdZipcode']+df['wtdDevice']+df['wtdIP&Phone']+df['wtdReg Date/Time']+df['wtdReferralCode']+df['wtdEmail'] 
df['Registration_Abuse_Score'] = (df['Registration_Abuse_Score']-df['Registration_Abuse_Score'].min())/(df['Registration_Abuse_Score'].max()-df['Registration_Abuse_Score'].min())
df['Registration_Abuse_Score'] = df['Registration_Abuse_Score'].apply(lambda x: x*100)

df['Registration_Abuse_Score']=df['Registration_Abuse_Score'].round(2)

In [170]:
df['Registration_Abuse_Score'] = np.where(df['Vowels']==0,100,df['Registration_Abuse_Score'])
df['Registration_Abuse_Score'] = np.where(df['UserCountsPerIP']>=20,100,df['Registration_Abuse_Score'])
df['Registration_Abuse_Score'] = np.where(df['PlusCharFlag']==1,100,df['Registration_Abuse_Score'])

In [171]:
df.columns.tolist()

['FirstName',
 'MiddleName',
 'LastName',
 'Dob',
 'Gender',
 'UserId',
 'Status',
 'Email',
 'Mobile',
 'ProgramId',
 'ProgramName',
 'PostalCode',
 'BadTryCount',
 'LastBadLoginTryTime',
 'LastLoginTime',
 'LastPinChangeTime',
 'PresentLoginTime',
 'RegisteredOn',
 'OS Type',
 'Reg Referral Code',
 'Reg Referral Prefix',
 'Ref PC AC Number',
 'Reg Client IP',
 'Device Id',
 'Sim Id',
 ' Meta Data',
 'FirstNameLower',
 'MiddleNameLower',
 'LastNameLower',
 'FullName',
 'Score',
 'IsGibberish',
 'FirstNameLen',
 'MiddleNameLen',
 'LastNameLen',
 'TotalNameLength',
 'EnglishWordsCount',
 'EnglishWords',
 'TotalWordsCount',
 'PctEnglishWordsCount',
 'Vowels',
 'MultipleSameNameReg',
 'RegMonth',
 'RegDay',
 'DobCleaned',
 'DobMonth',
 'DobDay',
 'IsBdayFlag(Month)',
 'IsBdayFlag(Day)',
 'LastLoginTime(sec)',
 'LastLoginTime(hr)',
 'LastLoginTime(min)',
 'RegisteredOn(Hour)',
 'RegisteredOn(Min)',
 'LoginTimeInHrs',
 'LoginTimeInMin',
 'AverageLoginTime(Hours)',
 'AverageLoginTime(Min)',


### Developing a reason code for the main driver leading to a high/low prime indicator 

In [172]:
#initialize a null string and keep on appending if the condition is true(csv)

In [173]:
#conditions  = [((df['Name'] > 60) & (df['Zipcode']>60)&(df['Email ']>60)),((df['Name']>60) & (df['Zipcode']>60)&(df['Reg Date/Time']>60)),((df['Name']>60) & (df['Zipcode']>60)&(df['Device']>60)),((df['Name']>60) & (df['Reg Date/Time']>60)),((df['Name']>60) & (df['Zipcode']>60)),((df['Name']>60) & (df['Email ']>60)),((df['Name']>60) & (df['ReferralCode']>60)),(df['Name']>60), (df['Zipcode']>60) ,(df['Email ']>60),((df['Reg Date/Time']>60)&(df['ReferralCode']>60)),((df['Reg Date/Time']>60)&(df['Zipcode']>60)),(df['Reg Date/Time']>60)]
#choices     = ['Name,Zipcode,Email','Name,Zipcode,Reg Date/Time','Name,Zipcode,Device','Name,Reg Date/Time','Name,Zipcode','Name,Email','Name,ReferralCode','Name','Zipcode','Email','Reg Date/Time,ReferralCode','Reg Date/Time,Zipcode','Reg Date/Time']
#df["Reason"] = np.select(conditions, choices, default='')

In [174]:
df['name'] = np.where(df['IsGibberish']==1,'Name','')

In [175]:
df['multiplereg'] = np.where(df['MultipleRegistrations']>=30,'Multiple_Registrations_With_Same_Name','')

In [176]:
df['zipcode'] = np.where(df['PostalCodeCheck ']==1.000,'Zipcode','')

In [177]:
df['email'] = np.where(df['Email ']>60,'Email','')

In [178]:
df['device'] = np.where(df['Device']>=45,'Device','')

In [179]:
df['reg']=np.where(df['Reg Date/Time']>=60,'Reg Date/Time','')

In [180]:
df['ip&ph']=np.where(df['IP&Phone']>50,'IP&Phone','')

In [181]:
df['ref']=np.where(df['ReferralCode']>60,'Referral Code','')

In [182]:
print(df.columns.tolist())

['FirstName', 'MiddleName', 'LastName', 'Dob', 'Gender', 'UserId', 'Status', 'Email', 'Mobile', 'ProgramId', 'ProgramName', 'PostalCode', 'BadTryCount', 'LastBadLoginTryTime', 'LastLoginTime', 'LastPinChangeTime', 'PresentLoginTime', 'RegisteredOn', 'OS Type', 'Reg Referral Code', 'Reg Referral Prefix', 'Ref PC AC Number', 'Reg Client IP', 'Device Id', 'Sim Id', ' Meta Data', 'FirstNameLower', 'MiddleNameLower', 'LastNameLower', 'FullName', 'Score', 'IsGibberish', 'FirstNameLen', 'MiddleNameLen', 'LastNameLen', 'TotalNameLength', 'EnglishWordsCount', 'EnglishWords', 'TotalWordsCount', 'PctEnglishWordsCount', 'Vowels', 'MultipleSameNameReg', 'RegMonth', 'RegDay', 'DobCleaned', 'DobMonth', 'DobDay', 'IsBdayFlag(Month)', 'IsBdayFlag(Day)', 'LastLoginTime(sec)', 'LastLoginTime(hr)', 'LastLoginTime(min)', 'RegisteredOn(Hour)', 'RegisteredOn(Min)', 'LoginTimeInHrs', 'LoginTimeInMin', 'AverageLoginTime(Hours)', 'AverageLoginTime(Min)', 'DeviationFromAvgLoginTime(Hrs)', 'DeviationFromAvgLoginT

In [183]:
df['PostalCode'] = df['PostalCode'].astype(str)

In [184]:
r = df.apply(lambda x: x.zipcode + ' ' + x.PostalCode, axis=1)
#df['zipcode'] = np.where(df['zipcode']== 'Zipcode nan','Zipcode -','')

In [185]:
df['zipcode'] = np.where(df['zipcode']=='Zipcode',r,'')

In [186]:
df['Reason'] = df[['name','multiplereg','zipcode','email','device','reg','ref','ip&ph']].apply(lambda x: ' '.join(x.fillna(' ').map(str)), axis=1)
df['Reason'] = df['Reason'].str.strip(' ')


In [187]:
df['Reason'].value_counts()

                                                     1075
Reg Date/Time                                          77
Name     Reg Date/Time  IP&Phone                       67
Name     Reg Date/Time                                 33
Reg Date/Time  IP&Phone                                30
                                                     ... 
Name  Zipcode 798987.0 Email Device Reg Date/Time       1
Name  Zipcode 565555.0 Email  Reg Date/Time             1
Name  Zipcode 556655.0 Email Device                     1
Zipcode 124578.0 Email Device Reg Date/Time             1
Name  Zipcode 491616.0 Email  Reg Date/Time             1
Name: Reason, Length: 141, dtype: int64

In [188]:
df = df.drop(['name','multiplereg',
      'zipcode', 'email', 'device', 'reg', 'ref','ip&ph','ScoreNorm', 'ScoreNormRev', 'GibberishFlagNorm', 'UsersPerMetaDataNorm', 'UserCountPerDeviceIdNorm',  'PostalCodeCheckNorm', 'IsBdayFlag(Month)Norm', 'IsBdayFlag(Day)Norm', 'DeviationLoginTime-HrNorm', 'DeviationLoginTime-MinNorm', 'UserCountsPerIPNorm', 'UsersPerBlock1Norm', 'UsersPerBlock2Norm', 'UsersPerBlock3Norm', 'CountryMatching', 'CountryMatchingNorm', 'StateMatchingNorm', 'LocalityMatchingNorm', 'ZipcodeMatch-UserEntered&IPNorm', 'RefCodesVelocityNorm','NumCharCountNorm','EmailTextCountNorm','SpclCharCountNorm','RefCodesVelocity'], axis=1)

### Dropping unecessary columns 

In [189]:
df = df.drop(['WtdScore','wtdMultipleReg', 'WtdGibFlag','CountryCodeMatching(Mobile)Norm', 'WtdPostalCodeCheckNorm', 'WtdUsersPerMetaDataNorm', 'WtdUserCountPerSimIdNorm', 'WtdUserCountPerDeviceIdNorm','WtdIsBdayFlag(Month)Norm', 'WtdIsBdayFlag(Day)Norm', 'WtdDeviationLoginTime-HrNorm', 'WtdDeviationLoginTime-MinNorm', 'WtdUserCountsPerIPNorm', 'WtdUsersPerBlock1Norm', 'WtdUsersPerBlock2Norm', 'WtdUsersPerBlock3Norm', 'WtdCountryMatchingNorm', 'WtdStateMatchingNorm', 'WtdLocalityMatchingNorm', 'WtdZipcodeMatch-UserEntered&IPNorm', 'WtdCountryCodeMatching(Mobile)Norm', 'WtdRefCodesVelocityNorm', 'wtdName', 'wtdZipcode', 'wtdDevice', 'wtdIP&Phone', 'wtdReg Date/Time', 'wtdReferralCode', 'wtdEmail','wtdName',
      'wtdZipcode', 'wtdDevice', 'wtdReg Date/Time', 'wtdReferralCode','wtdEmail','wtdIP&Phone',"PlusCharFlagNorm"], axis =1)

In [190]:
df = df.rename(columns={'Unnamed: 0':'Sno.'})

In [191]:
#df = df[['Sno.', 'FirstName', 'MiddleName', 'LastName', 'Dob', 'Gender', 'UserId', 'Status', 'Email', 'phone_numbers', 'ProgramId', 'ProgramName', 'PostalCode', 'BadTryCount', 'LastBadLoginTryTime', 'LastLoginTime', 'LastPinChangeTime', 'PresentLoginTime', 'RegisteredOn', 'OS Type', 'Reg Referral Code', 'Reg Referral Prefix', 'Ref PC AC Number', 'ip', 'Device Id', 'Sim Id', ' Meta Data', 'FirstNameLower', 'MiddleNameLower', 'LastNameLower', 'FullName', 'Score', 'IsGibberish', 'FirstNameLen', 'MiddleNameLen', 'LastNameLen', 'TotalNameLength', 'EnglishWordsCount', 'EnglishWords', 'TotalWordsCount', 'PctEnglishWordsCount', 'RegisteredOn-Month', 'RegisteredOn-Day', 'Dob*', 'Dob-Month', 'Dob-Day', 'IsBdayFlag(Month)', 'IsBdayFlag(Day)', 'LastLoginTime(sec)', 'LastLoginTime(hr)', 'LastLoginTime(min)', 'RegisteredOn(Hour)', 'RegisteredOn(Min)', 'LoginTimeInHrs', 'LoginTimeInMin', 'AverageLoginTime(Hours)', 'AverageLoginTime(Min)', 'DeviationFromAvgLoginTime(Hrs)', 'DeviationFromAvgLoginTime(Min)', 'DeviationLoginTime-Hr', 'DeviationLoginTime-Min', 'UsersPerMetaData', 'UsersPerEmail', 'Domain', 'e', 'EmailCheck', 'DomainCount', 'PostalCodeCounts', 'PostalCodeCheck', 'regionname', 'circlename', 'taluk', 'districtname', 'statename', 'PostalCountsPerRef', 'UserCountsPerIP', 'UsersPerAccount', 'IPBlock1', 'UsersPerBlock1', 'IPBlock2', 'UsersPerBlock2', 'IPBlock3', 'UsersPerBlock3', 'UserCountPerDeviceId', 'UserCountPerSimId', 'RefCodesVelocity', 'country_codeip', 'country_name', 'region_name', 'city_name', 'zip_code', 'phonenumber', 'is_valid', 'number', 'local_format', 'international_format', 'country_prefix', 'country_code', 'location', 'carrier', 'line_type', 'json', 'CountryMatching(IP)', 'CountryCodeMatching(IP)', 'CountryCodeMatching(Mobile)', 'ExtractedPinDigits', 'ExtractedPinDigits(IP)', 'StateMatching', 'LocalityMatching', 'UserEnteredZipCodeMatchingWithIPDerivedZipCode', 'ScoreNorm', 'ScoreNormRev', 'GibberishFlagNorm', 'UsersPerMetaDataNorm', 'UserCountPerDeviceIdNorm', 'PostalCodeCountsNorm', 'PostalCodeCheckNorm', 'PostalCountsPerRefNorm', 'IsBdayFlag(Month)Norm', 'IsBdayFlag(Day)Norm', 'DeviationLoginTime-HrNorm', 'DeviationLoginTime-MinNorm', 'UserCountsPerIPNorm', 'UsersPerBlock1Norm', 'UsersPerBlock2Norm', 'UsersPerBlock3Norm', 'CountryMatching', 'CountryMatchingNorm', 'StateMatchingNorm', 'LocalityMatchingNorm', 'UserEnteredZipCodeMatchingWithIPDerivedZipCodeNorm', 'RefCodesVelocityNorm', 'WtdScore', 'WtdGibFlag', 'WtdPostalCodeCountsNorm', 'WtdPostalCodeCheckNorm', 'WtdPostalCountsPerRefNorm', 'WtdUsersPerMetaDataNorm', 'WtdUserCountPerSimIdNorm', 'WtdUserCountPerDeviceIdNorm', 'WtdIsBdayFlag(Month)Norm', 'WtdIsBdayFlag(Day)Norm', 'WtdDeviationLoginTime-HrNorm', 'WtdDeviationLoginTime-MinNorm', 'WtdUserCountsPerIPNorm', 'WtdUsersPerBlock1Norm', 'WtdCountryMatchingNorm', 'WtdStateMatchingNorm', 'WtdLocalityMatchingNorm', 'WtdUserEnteredZipCodeMatchingWithIPDerivedZipCodeNorm', 'WtdRefCodesVelocityNorm', 'wtdName', 'wtdZipcode', 'wtdDevice', 'wtdReg Date/Time', 'wtdReferralCode','Name','Zipcode','Device','Reg Date/Time','IP','ReferralCode','PrimeIndicator','Reason']]
df.shape

(1485, 146)

In [192]:
#from matplotlib.pyplot import figure
#df=df.sort_values('Registration_Abuse_Score')
#sns.distplot(df['Registration_Abuse_Score'])
#df['AbuseStatus'] = np.where(df['Registration_Abuse_Score']>60,1,0)
#df['Decile_rank'] = pd.qcut(df['Registration_Abuse_Score'], 10, 
                            #labels = False) 
#df['Decile_rank'].value_counts()
#sns.boxplot(df['Decile_rank'])
#plt.figure(figsize=(10,15))
#df['Percentile_rank'].value_counts(normalize=True).plot.bar()
#import seaborn as sns
#sns.boxplot(df['Registration_Abuse_Score'])
#sns.boxplot(df['Percentile_rank'])
#plt.figure(figsize=(10,15))
#df['Registration_Abuse_Score'].value_counts(normalize=True).plot.bar()
#df['Registration_Abuse_Score_Decile'] = pd.qcut(df['Percentile_rank'], 11, labels = False) 

In [193]:
df['Registration_Abuse_Score']

0        7.99
1       10.44
2       10.63
3       16.95
4       14.49
        ...  
1480    16.87
1481    13.73
1482     5.04
1483     5.17
1484     4.24
Name: Registration_Abuse_Score, Length: 1485, dtype: float64

In [194]:
#df['Percentile_rank'] = pd.qcut(df['Registration_Abuse_Score'], 101, labels = False) 

In [195]:
''''rank=[]
for value in df['Percentile_rank']:
    if value<=10:
        rank.append('0-10')
    elif value>10 and value<=20:
        rank.append('11-20')
    elif value>20 and value<=30:
        rank.append('21-30')
    elif value>30 and value<=40:
        rank.append('31-40')
    elif value>40 and value<=50:
        rank.append('41-50')
    elif value>50 and value<=60:
        rank.append('51-60')
    elif value>60 and value<=70:
        rank.append('61-70')
    elif value>70 and value<=80:
        rank.append('71-80')
    elif value>80 and value<=90:
        rank.append('81-90')    
    else:
        rank.append('91-100')
df['Registration_Abuse_Score_Decile'] =rank  '''  

"'rank=[]\nfor value in df['Percentile_rank']:\n    if value<=10:\n        rank.append('0-10')\n    elif value>10 and value<=20:\n        rank.append('11-20')\n    elif value>20 and value<=30:\n        rank.append('21-30')\n    elif value>30 and value<=40:\n        rank.append('31-40')\n    elif value>40 and value<=50:\n        rank.append('41-50')\n    elif value>50 and value<=60:\n        rank.append('51-60')\n    elif value>60 and value<=70:\n        rank.append('61-70')\n    elif value>70 and value<=80:\n        rank.append('71-80')\n    elif value>80 and value<=90:\n        rank.append('81-90')    \n    else:\n        rank.append('91-100')\ndf['Registration_Abuse_Score_Decile'] =rank  "

In [197]:
df.to_excel('Registration_Score1.xlsx', index=False)

In [198]:
df.shape

(1485, 146)